In [1]:
from nltk.translate.bleu_score import sentence_bleu
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import json

In [2]:
# Load the merged dataset and the results from finetuning
df_merged = pd.read_csv(
    '../data/parsed/reports_promet_merged.csv',
    parse_dates=['report_datetime', 'info_datetime']
)

with open('../generated_reports_finetuning.json', 'r', encoding='utf-8') as f:
    results_finetuning = json.load(f)

In [3]:
bleu_scores_finetuned = []
bleu_scores_baseline = []

for res in results_finetuning:
    # Tokenize the actual and generated reports
    actual = res['actual_report']
    generated = res['generated_report']
    actual_tokens = actual.split()
    generated_tokens = generated.split()

    # Get the entry from the merged dataset and take the raw info as the baseline
    mask = df_merged.report_content.str.strip() == actual
    row = df_merged[mask].head(1).iloc[0]
    baseline = ''
    if not pd.isna(row['info_aggregated_urgent']): baseline += row['info_aggregated_urgent'] + '\n'
    if not pd.isna(row['info_aggregated_basic']): baseline += row['info_aggregated_basic'] + '\n'
    basline_tokens = baseline.split()

    # Compute BLEU score
    bleu_fine = sentence_bleu([actual_tokens], generated_tokens)
    blue_base = sentence_bleu([actual_tokens], basline_tokens)
    bleu_scores_baseline.append(blue_base)
    bleu_scores_finetuned.append(bleu_fine)

df_bleu = pd.DataFrame({
    'Fine-Tuned': bleu_scores_finetuned,
    'Baseline': bleu_scores_baseline
})

df_bleu.mean().to_frame('Mean BLEU Score')

,Mean BLEU Score
Fine-Tuned,0.074468
Baseline,0.072162
